In [2]:
import numpy as np
import cv2 as cv
import glob

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*7,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:7].T.reshape(-1,2) * 75

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/*.jpg')

#print(images)

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9,7), None)
    print(fname, ret)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (9,7), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0000.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0050.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0100.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0150.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0200.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0250.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0300.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0350.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0400.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0450.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0500.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0550.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0600.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0650.jpg True
/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_0700.jpg 

In [3]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(ret)

0.3640913844936302


In [8]:
img = cv.imread('/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frames/frame_2600.jpg')
h,  w = img.shape[:2]
print(h, w)
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

1080 1920


In [9]:
# Undistort the image
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# Save the entire undistorted image without cropping
cv.imwrite('/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/frame_2600_und_2.jpg', dst)


True

For video

In [4]:
video_path = '/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/test_cal.mp4'
cap = cv.VideoCapture(video_path)


#here we get video properties, reading the video frame by frame, out of it we get:
# - width of frame in pixel
# - height of frame in pixel
# - frames per seconds of the video
#video properties are then used because the undistorted video needs to match

frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv.CAP_PROP_FPS)

#here we define the options for saving the video output
fourcc = cv.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv.VideoWriter('/Volumes/COCOZZA 2^D/DATA/0403_T1p5_H20/test_cal_und.MP4', fourcc, fps, (frame_width, frame_height))

In [5]:
#here we optimize the camera matrix for undistortion
#as input it gets the original caamera metrix obtained from calibration as well as the distortion coefficients from calibration
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (frame_width, frame_height), 1, (frame_width, frame_height))


In [6]:
#here we read each frame, undistort it, write into an output video file

#compute transformation maps (is the same for all frames, so out of the loop)
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (frame_width, frame_height), 5)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stops if video ends
    
#undistort frame by using the pre computed maps
    undistorted_frame = cv.remap(frame, mapx, mapy, cv.INTER_LINEAR)

#write frame to output video
    out.write(undistorted_frame)


    cv.imshow('Undistorted Video', undistorted_frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
out.release()
cv.destroyAllWindows()

In [7]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    mean_error += error

print(f"Reprojection Error: {mean_error / len(objpoints)}")


Reprojection Error: 0.02935961380415039
